# 🛠️ Custom Transformers: Twoje własne klocki w Pipeline

Scikit-Learn pozwala Ci budować własne klasy, które zachowują się dokładnie tak jak `StandardScaler` czy `PCA`.
Dzięki temu możesz wrzucić swoją unikalną logikę biznesową (np. czyszczenie walut, wyciąganie danych z daty) prosto do Pipeline'a.

Aby to zrobić, musimy stworzyć klasę, która dziedziczy po:
1.  **`BaseEstimator`**: Pozwala używać naszej klasy w GridSearchu (automatycznie obsługuje parametry).
2.  **`TransformerMixin`**: Daje nam darmową metodę `.fit_transform()`.

Musimy napisać tylko dwie metody:
*   `fit(self, X, y=None)`: Czego model ma się nauczyć z danych? (Np. średniej). Jeśli niczego (bo tylko czyścimy tekst) -> zwracamy `self`.
*   `transform(self, X)`: Tu dzieje się właściwa robota (zmiana danych).

In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# 1. BRUDNE DANE
# Wyobraź sobie dane prosto ze sklepu internetowego (scraping).
data = pd.DataFrame({
    'Produkt': ['Laptop', 'Myszka', 'Klawiatura', 'Monitor'],
    'Cena_Brudna': ['$1,200.50', '$50.00', '$150.99', '$300.00'], # To są napisy!
    'Waga_Opis': ['Lekki (1.5kg)', 'Bardzo lekki (0.1kg)', 'Średni (0.8kg)', 'Ciężki (5.0kg)'],
    'Sprzedaz': [100, 500, 200, 50] # To chcemy przewidzieć
})

print("--- DANE WEJŚCIOWE (Napisy zamiast liczb!) ---")
display(data)
print(data.dtypes)

--- DANE WEJŚCIOWE (Napisy zamiast liczb!) ---


,Produkt,Cena_Brudna,Waga_Opis,Sprzedaz
0,Laptop,"$1,200.50",Lekki (1.5kg),100
1,Myszka,$50.00,Bardzo lekki (0.1kg),500
2,Klawiatura,$150.99,Średni (0.8kg),200
3,Monitor,$300.00,Ciężki (5.0kg),50


Produkt        object
Cena_Brudna    object
Waga_Opis      object
Sprzedaz        int64
dtype: object


## Transformator 1: CurrencyCleaner

Napiszemy klasę, która:
1.  Bierze kolumnę tekstową.
2.  Usuwa znaki `$` i `,`.
3.  Zamienia resztę na `float`.

Zauważ, że w metodzie `fit` nic nie robimy (`return self`), bo czyszczenie dolara nie wymaga "uczenia się" od danych. To operacja bezstanowa.

In [2]:
class CurrencyCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        # Nic do nauczenia. Nie liczymy średniej ani nic.
        # Po prostu zwracamy obiekt, żeby Pipeline się nie zaciął.
        return self
    
    def transform(self, X):
        # Kopiujemy dane, żeby nie psuć oryginału (Dobra praktyka!)
        X_copy = X.copy()
        
        # Logika czyszczenia
        # 1. Usuń '$'
        # 2. Usuń ',' (dla tysięcy)
        # 3. Zamień na float
        X_copy[self.column_name] = X_copy[self.column_name].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
        
        return X_copy

# Przetestujmy to ręcznie
cleaner = CurrencyCleaner(column_name='Cena_Brudna')
data_clean = cleaner.fit_transform(data)

print("--- PO CZYSZCZENIU WALUTY ---")
display(data_clean)
print(data_clean.dtypes)

--- PO CZYSZCZENIU WALUTY ---


,Produkt,Cena_Brudna,Waga_Opis,Sprzedaz
0,Laptop,1200.50,Lekki (1.5kg),100
1,Myszka,50.00,Bardzo lekki (0.1kg),500
2,Klawiatura,150.99,Średni (0.8kg),200
3,Monitor,300.00,Ciężki (5.0kg),50


Produkt         object
Cena_Brudna    float64
Waga_Opis       object
Sprzedaz         int64
dtype: object


## Transformator 2: WeightExtractor (Regex)

Teraz trudniejsze zadanie. Mamy kolumnę `Waga_Opis`: "Lekki (1.5kg)".
Chcemy wyciągnąć samą liczbę `1.5` z nawiasu.

Użyjemy wyrażeń regularnych (Regex).

In [3]:
class WeightExtractor(BaseEstimator, TransformerMixin):
    def transform(self, X, y=None):
        X_copy = X.copy()
        
        # Regex: Szukamy cyfr, kropki, cyfr wewnątrz nawiasu
        # '(\d+\.?\d*)kg' oznacza: znajdź liczbę przed 'kg'
        X_copy['Waga_Liczbowa'] = X_copy['Waga_Opis'].str.extract(r'(\d+\.?\d*)').astype(float)
        
        # Usuwamy starą kolumnę tekstową (bo model jej nie zrozumie)
        return X_copy.drop(columns=['Waga_Opis', 'Produkt']) # Usuwamy też Produkt bo to tekst

    def fit(self, X, y=None):
        return self

# Test
extractor = WeightExtractor()
data_final = extractor.fit_transform(data_clean)

print("--- PO EKSTRAKCJI WAGI ---")
display(data_final)

--- PO EKSTRAKCJI WAGI ---


,Cena_Brudna,Sprzedaz,Waga_Liczbowa
0,1200.50,100,1.5
1,50.00,500,0.1
2,150.99,200,0.8
3,300.00,50,5.0


## Wielki Finał: Pipeline

Złóżmy to w całość.
Nasz Pipeline weźmie te brudne dane, wyczyści dolary, wyciągnie wagę z nawiasów i na końcu wytrenuje Regresję.

In [4]:
# Definiujemy pełny proces
moj_pipeline = Pipeline([
    # Krok 1: Wyczyść cenę
    ('cleaner', CurrencyCleaner(column_name='Cena_Brudna')),
    
    # Krok 2: Wyciągnij wagę i usuń zbędny tekst
    ('extractor', WeightExtractor()),
    
    # Krok 3: Model
    ('model', LinearRegression())
])

# Dzielimy na X i y
X = data[['Produkt', 'Cena_Brudna', 'Waga_Opis']] # Brudne wejście
y = data['Sprzedaz']

# TRENING (Magia!)
# Pipeline sam uruchomi fit() i transform() po kolei
moj_pipeline.fit(X, y)

print("✅ Pipeline wytrenowany na brudnych danych!")

# PREDYKCJA
# Wyobraź sobie, że to nowe dane z API
nowe_dane = pd.DataFrame({
    'Produkt': ['Tablet'],
    'Cena_Brudna': ['$450.00'],
    'Waga_Opis': ['Super (0.4kg)']
})

predykcja = moj_pipeline.predict(nowe_dane)
print(f"\nNowy produkt: {nowe_dane.iloc[0].values}")
print(f"Przewidywana sprzedaż: {predykcja[0]:.2f} sztuk")

✅ Pipeline wytrenowany na brudnych danych!

Nowy produkt: ['Tablet' '$450.00' 'Super (0.4kg)']
Przewidywana sprzedaż: 300.93 sztuk


## 🧠 Podsumowanie: Dlaczego warto pisać klasy?

Mogliśmy to zrobić funkcją `lambda` w Pandas, prawda?
Tak, ale...

**Tu jest haczyk (Produkcja i Serializacja).**
1.  Gdybyś użył funkcji lambda, musiałbyś kopiować ten kod do pliku `api.py` na serwerze.
2.  Używając klasy `CurrencyCleaner`, możesz zapisać cały `moj_pipeline` do pliku `.pkl` (Pickle).
3.  Kiedy wczytasz ten plik na serwerze, **cała logika czyszczenia jest w środku modelu**.

Wrzucasz brudne dane -> Wychodzi wynik.
To jest definicja **Robust ML System**. Twój model jest "samowystarczalny".